# Análisis Exploratorio de Datos (EDA) - Online Retail

En este cuaderno, realizaremos un **análisis paso a paso** de un dataset de facturación que contiene columnas como:
 - **InvoiceNo**: Número de factura
 - **StockCode**: Código del producto
 - **Description**: Descripción del producto
 - **Quantity**: Cantidad vendida (puede ser negativa en devoluciones)
 - **InvoiceDate**: Fecha de la factura
 - **UnitPrice**: Precio unitario de cada artículo
 - **CustomerID**: ID del cliente (puede haber nulos si no se registró)
 - **Country**: País donde se realizó la compra

El objetivo es **entender** el comportamiento de las ventas, los países más relevantes, los productos más comprados, y otras métricas de interés.

In [63]:
!wget https://github.com/javierherrera1996/lecture_analytics/raw/main/datasets/ecomerce_customers_segmentation.zip
!unzip ecomerce_customers_segmentation.zip

--2025-03-06 00:21:08--  https://github.com/javierherrera1996/lecture_analytics/raw/main/datasets/ecomerce_customers_segmentation.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/javierherrera1996/lecture_analytics/main/datasets/ecomerce_customers_segmentation.zip [following]
--2025-03-06 00:21:09--  https://raw.githubusercontent.com/javierherrera1996/lecture_analytics/main/datasets/ecomerce_customers_segmentation.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7548686 (7.2M) [application/zip]
Saving to: ‘ecomerce_customers_segmentation.zip.1’

ecomerce_customers_ 100%[===================

## 1. Importar librerías y cargar datos
Asegúrate de tener el archivo CSV en la misma carpeta, o ajusta la ruta de `pd.read_csv` según tu necesidad.

In [80]:
import pandas as pd
import numpy as np

# Carga del dataset (ajusta el nombre a tu archivo)
df = pd.read_csv('data.csv', encoding='latin1')  # Puedes necesitar encoding='latin1' si hay caracteres especiales

# Mostramos las primeras filas para inspección
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


## 2. Información general del dataset
Aquí veremos:
- Dimensiones del DataFrame
- Tipos de datos
- Presencia de valores nulos
- Estadísticos descriptivos de las columnas numéricas

In [81]:
# Dimensiones (filas, columnas)
df.shape

(541909, 8)

In [82]:
# Información general
type(df)

pandas.core.frame.DataFrame

In [83]:
# Estadísticos descriptivos
df.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


**Observación**: Algunas facturas pueden ser devoluciones si `Quantity` es negativa. Además, puede haber filas sin `CustomerID`.

## 3. Limpieza básica y transformación
### 3.1. Revisar y eliminar posibles filas con datos problemáticos
 - Filas donde `Description` esté vacía
 - Filas con `Quantity == 0` (puede haber poco sentido en un registro así)
 - Filas con `UnitPrice <= 0` (podría ser error o promoción anómala)
 - Convertir la columna `InvoiceDate` a formato fecha, para luego analizar tiempos

In [84]:
# Eliminamos filas donde no haya descripción
df = df[df['Description'] != '']

# Eliminamos filas donde Quantity sea 0
df = df[df['Quantity'] != 0]

# Eliminamos filas donde UnitPrice sea menor o igual a 0
df = df[df['Quantity'] > 0]

# Verificamos resultado
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


## 4. Creación de columna "TotalPrice"
Para conocer el valor total de cada línea de venta, calculamos `Quantity * UnitPrice`.

In [85]:
TotalPrice = df['Quantity'] * df['UnitPrice']
df['TotalPrice'] = TotalPrice
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34


## 5. Análisis paso a paso
A continuación, veremos distintos análisis que **paso a paso** nos ayudarán a entender mejor el negocio.

### 5.1. ¿Cuántas transacciones totales hay y cuántos clientes diferentes?
Esto da un primer vistazo al tamaño del negocio en términos de facturas y base de clientes.

In [40]:
TransactionsNumber = df['InvoiceNo'].count()
UniqueCustomers = df ['CustomerID'].nunique()
print(f"Total Transactions: {TransactionsNumber}")
print(f"Unique Customers: {UniqueCustomers}")

Total Transactions: 541909
Unique Customers: 4372


### 5.2. ¿Cuáles son los países más frecuentes en las ventas?
Podemos contar cuántas líneas de factura vienen de cada país. Esto ayuda a entender nuestro alcance geográfico.

In [86]:
TopCountries = df.groupby('Country')['TotalPrice'].sum().sort_values(ascending=False)
print(TopCountries)

Country
United Kingdom          9003097.964
Netherlands              285446.340
EIRE                     283453.960
Germany                  228867.140
France                   209715.110
Australia                138521.310
Spain                     61577.110
Switzerland               57089.900
Belgium                   41196.340
Sweden                    38378.330
Japan                     37416.370
Norway                    36165.440
Portugal                  33747.100
Finland                   22546.080
Singapore                 21279.290
Channel Islands           20450.440
Denmark                   18955.340
Italy                     17483.240
Hong Kong                 15691.800
Cyprus                    13590.380
Austria                   10198.680
Israel                     8135.260
Poland                     7334.650
Greece                     4760.520
Unspecified                4749.790
Iceland                    4310.000
Canada                     3666.380
USA                 

> Tip: Si queremos ver el total en **monetario** por país, podemos agrupar por `Country` y sumar `TotalPrice`.

## 5.3. ¿Cuáles son los productos más vendidos?
Aquí vamos a usar Description para ver cuántas líneas se han vendido (o la suma total de Quantity).

In [89]:
# Top 10 productos por cantidad total vendida
TopProducts = df.groupby('Description')['Quantity'].sum().sort_values(ascending=False).head(10)
print(TopProducts)

Description
PAPER CRAFT , LITTLE BIRDIE           80995
MEDIUM CERAMIC TOP STORAGE JAR        78033
WORLD WAR 2 GLIDERS ASSTD DESIGNS     55047
JUMBO BAG RED RETROSPOT               48478
WHITE HANGING HEART T-LIGHT HOLDER    37895
POPCORN HOLDER                        36761
ASSORTED COLOUR BIRD ORNAMENT         36461
PACK OF 72 RETROSPOT CAKE CASES       36419
RABBIT NIGHT LIGHT                    30788
MINI PAINT SET VINTAGE                26633
Name: Quantity, dtype: int64


### 5.4. ¿Cuál es el total de ingresos (revenue) y el ticket promedio?
 - El ingreso total es la suma de **TotalPrice**.
 - El ticket promedio podemos calcularlo como el promedio de **TotalPrice** dentro de cada factura (o línea).

In [90]:
TotalSum= df['TotalPrice'].sum()
TotalAverage= df['TotalPrice'].mean()
print(f"Total Sum: {TotalSum}")
print(f"Total Average: {TotalAverage}")

Total Sum: 10644560.424
Total Average: 20.035499635788703


> Para un ticket promedio a nivel de **factura**, debemos agrupar por `InvoiceNo`, sumar `TotalPrice` y luego promediar.

In [91]:
# Ticket promedio por factura
TicketAverage = df.groupby('InvoiceNo')['TotalPrice'].sum()
print(TicketAverage)

InvoiceNo
536365       139.12
536366        22.20
536367       278.73
536368        70.05
536369        17.85
             ...   
581586       339.20
581587       249.45
A563185    11062.06
A563186   -11062.06
A563187   -11062.06
Name: TotalPrice, Length: 20728, dtype: float64


### 5.6. ¿Tenemos devoluciones?
Como `Quantity` puede ser negativa, **representa** devoluciones. Veamos cuántas hay y el impacto en valor.

In [92]:
# Filtrar devoluciones (Quantity < 0)
df[df['Quantity'] < 0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice


### 5.7. ¿Cuántos clientes realizan compras repetidas?
Podemos ver cuántos clientes compran más de una vez.

In [93]:
# Agrupamos por CustomerID y contamos facturas únicas
RepeatedPurchase = df.groupby('CustomerID')['InvoiceNo'].nunique()
print(RepeatedPurchase)

CustomerID
12346.0     1
12347.0     7
12348.0     4
12349.0     1
12350.0     1
           ..
18280.0     1
18281.0     1
18282.0     2
18283.0    16
18287.0     3
Name: InvoiceNo, Length: 4339, dtype: int64


## 6. Posibles siguientes pasos
- **Crear segmentaciones** (p. ej. RFM: Recency, Frequency, Monetary) para clasificar clientes.
- **Analizar descuentos/promos** si tuvieras columnas extra.
- **Fusionar** con datos de marketing para calcular CAC (Customer Acquisition Cost) y LTV (Lifetime Value), etc.
- **Explorar** variables estacionales o días de la semana con mayor venta.

Con este **análisis paso a paso**, se obtiene un primer panorama de:
 - Cantidad de facturas y clientes.
 - Productos más vendidos.
 - Ingresos por país.
 - Montos totales y tickets promedio.
 - Distribución mensual de ventas.
 - Devoluciones y recurrencia de clientes.

¡A medida que se adquieran más habilidades en Python y análisis de datos, se pueden añadir gráficos, segmentaciones avanzadas y modelos predictivos!